# demo: TextGraph + LLMs

## intro

In [1]:
import pathlib
from IPython.display import display, HTML

In [2]:
display(HTML(pathlib.Path("docs/demo/blurb.1.html").read_text(encoding = "utf-8")))

## parse a document

In [3]:
from icecream import ic
import pandas as pd
import pyvis
import spacy

from textgraph import Edge, Node, RenderPyVis, TextGraph

/Users/paco/src/textgraph/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
SRC_TEXT: str = """                                                                                                                      
Werner Herzog is a remarkable filmmaker and an intellectual originally from Germany, the son of Dietrich Herzog.
Although in fact Werner Herzog never really liked his father, as we all know.
After the war, Werner fled to America to become famous.
"""

In [5]:
tg: TextGraph = TextGraph()

sample_doc: spacy.tokens.doc.Doc = tg.build_doc(
    SRC_TEXT.strip(),
)

2023-11-29 07:04:10,061 - root - INFO - Initializing word embedding with word2vec.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
spacy.displacy.render(
    sample_doc,
    style = "ent",
    jupyter = True,
)

In [7]:
spacy.displacy.render(
    sample_doc,
    style = "dep",
    jupyter = True,
)

## build a lemma graph from the document

In [8]:
tg.build_graph_embeddings(
    sample_doc,
    debug = True,
)

ic| sent_id: 0
    sent: Werner Herzog is a remarkable filmmaker and an intellectual originally from Germany, the son of Dietrich Herzog.
    sent.start: 0
ic| sent_nodes: [Node(node_id=0, key='werner herzog.PROPN', span=Werner Herzog, text='Werner Herzog', pos='PROPN', sents={0}, kind='PERSON', count=1, neighbors=0, weight=0.0),
                 Node(node_id=1, key='1.is.AUX', span=is, text='is', pos='AUX', sents={0}, kind=None, count=0, neighbors=0, weight=0.0),
                 Node(node_id=2, key='2.a.DET', span=a, text='a', pos='DET', sents={0}, kind=None, count=0, neighbors=0, weight=0.0),
                 Node(node_id=3, key='3.remarkable.ADJ', span=remarkable, text='remarkable', pos='ADJ', sents={0}, kind=None, count=0, neighbors=0, weight=0.0),
                 Node(node_id=4, key='filmmaker.NOUN', span=filmmaker, text='filmmaker', pos='NOUN', sents={0}, kind=None, count=1, neighbors=0, weight=0.0),
                 Node(node_id=5, key='5.and.CCONJ', span=and, text='and', pos=

In [9]:
tg.calc_phrase_ranks()

ic(tg.edges);

ic| tg.edges: {'0.1.nsubj.0': Edge(src_node=0, dst_node=1, kind=<RelEnum.DEP: 0>, rel='nsubj', prob=1.0, count=1),
               '0.19.nsubj.0': Edge(src_node=0, dst_node=19, kind=<RelEnum.DEP: 0>, rel='nsubj', prob=1.0, count=1),
               '10.9.pobj.0': Edge(src_node=10, dst_node=9, kind=<RelEnum.DEP: 0>, rel='pobj', prob=1.0, count=1),
               '11.10.punct.0': Edge(src_node=11, dst_node=10, kind=<RelEnum.DEP: 0>, rel='punct', prob=1.0, count=1),
               '12.13.det.0': Edge(src_node=12, dst_node=13, kind=<RelEnum.DEP: 0>, rel='det', prob=1.0, count=1),
               '13.10.appos.0': Edge(src_node=13, dst_node=10, kind=<RelEnum.DEP: 0>, rel='appos', prob=1.0, count=1),
               '14.13.prep.0': Edge(src_node=14, dst_node=13, kind=<RelEnum.DEP: 0>, rel='prep', prob=1.0, count=1),
               '15.14.pobj.0': Edge(src_node=15, dst_node=14, kind=<RelEnum.DEP: 0>, rel='pobj', prob=1.0, count=1),
               '16.1.punct.0': Edge(src_node=16, dst_node=1, kind=

show the resulting entities extracted from the document

In [10]:
df: pd.DataFrame = tg.get_phrases_as_df()
df

,node_id,text,pos,kind,count,weight
0,0,Werner Herzog,PROPN,PERSON,2,0.100899
1,10,Germany,PROPN,GPE,1,0.099868
2,15,Dietrich Herzog,PROPN,PERSON,1,0.095201
3,37,Werner,PROPN,PERSON,1,0.095201
4,40,America,PROPN,GPE,1,0.095201
5,4,filmmaker,NOUN,None,1,0.088443
6,7,intellectual,NOUN,None,1,0.088443
7,13,son,NOUN,None,1,0.086235
8,25,father,NOUN,None,1,0.084152
9,35,war,NOUN,None,1,0.084152


## visualize the lemma graph

In [11]:
render: RenderPyVis = RenderPyVis(
    tg.nodes,
    tg.edges,
    tg.lemma_graph,
)

pv_graph: pyvis.network.Network = render.build_lemma_graph()

ic| node.count: 2
    node: Node(node_id=0, key='werner herzog.PROPN', span=Werner Herzog, text='Werner Herzog', pos='PROPN', sents={0, 1}, kind='PERSON', count=2, neighbors=2, weight=0.10089896719299135)
    nx_node: {'color': 'hsl(65, 46%, 58%)',
              'kind': <NodeKind.ENT: 2>,
              'label': 'Werner Herzog',
              'neighbors': 2,
              'shape': 'circle',
              'size': 2,
              'title': 'werner herzog.PROPN',
              'value': 0.10089896719299135}
ic| node.count: 0
    node: Node(node_id=1, key='1.is.AUX', span=is, text='is', pos='AUX', sents={0}, kind=None, count=0, neighbors=0, weight=0.0)
    nx_node: {'color': 'hsla(72, 19%, 90%, 0.4)',
              'kind': <NodeKind.DEP: 0>,
              'label': '',
              'neighbors': 0,
              'shape': 'star',
              'size': 0,
              'title': '1.is.AUX',
              'value': 0.0}
ic| node.count: 0
    node: Node(node_id=2, key='2.a.DET', span=a, text='a', p

set the layout parameters

In [12]:
pv_graph.force_atlas_2based(
    gravity = -38,
    central_gravity = 0.01,
    spring_length = 231,
    spring_strength = 0.7,
    damping = 0.8,
    overlap = 0,
)

pv_graph.show_buttons(filter_ = [ "physics" ])
pv_graph.toggle_physics(True)

In [13]:
pv_graph.prep_notebook()
pv_graph.show("vis.html")

vis.html


## cluster communities in the lemma graph

In [14]:
import networkx as nx

comm_iter = nx.community.girvan_newman(
    tg.lemma_graph,
)

top_level = next(comm_iter)
next_level = next(comm_iter)
communities = sorted(map(sorted, next_level))

print(f"{len(communities)} communities")
ic(communities)

ic| communities: [[0, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
                  [1, 2, 3, 4, 5, 6, 7, 16, 17],
                  [8, 9, 10, 11, 12, 13, 14, 15],
                  [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]]


4 communities


[[0, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
 [1, 2, 3, 4, 5, 6, 7, 16, 17],
 [8, 9, 10, 11, 12, 13, 14, 15],
 [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]]

In [15]:
n_list = list(tg.nodes.values())

for comm in communities:
    ic(comm)

    for node_id in comm:
        ic(node_id, n_list[node_id].text)

ic| comm: [0, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
ic| node_id: 0, n_list[node_id].text: 'Werner Herzog'
ic| node_id: 18, n_list[node_id].text: 'Although'
ic| node_id: 19, n_list[node_id].text: 'in'
ic| node_id: 20, n_list[node_id].text: 'fact'
ic| node_id: 21, n_list[node_id].text: 'never'
ic| node_id: 22, n_list[node_id].text: 'really'
ic| node_id: 23, n_list[node_id].text: 'liked'
ic| node_id: 24, n_list[node_id].text: 'his'
ic| node_id: 25, n_list[node_id].text: 'father'
ic| node_id: 26, n_list[node_id].text: ','
ic| node_id: 27, n_list[node_id].text: 'as'
ic| node_id: 28, n_list[node_id].text: 'we'
ic| node_id: 29, n_list[node_id].text: 'all'
ic| node_id: 30, n_list[node_id].text: 'know'
ic| node_id: 31, n_list[node_id].text: '.'
ic| node_id: 32
    n_list[node_id].text: '
                           '
ic| comm: [1, 2, 3, 4, 5, 6, 7, 16, 17]
ic| node_id: 1, n_list[node_id].text: 'is'
ic| node_id: 2, n_list[node_id].text: 'a'
ic| node_id: 3, n_list[node_id].te

## infer relations

In [16]:
if False:
    tg.infer_relations(
        SRC_TEXT.strip(),
        debug = True,
    )

## outro

_\[ more parts are in progress, getting added to this demo \]_